# COVID-19: Confirmed Total Cases vs Active Cases
**Data Sources:** 
1. https://datahub.io/core/covid-19#data
2. https://github.com/CSSEGISandData/COVID-19

In [1]:
import pandas as pd
import requests
from io import StringIO
from plotly.subplots import make_subplots
import plotly.graph_objects as go

url = 'https://datahub.io/core/covid-19/r/time-series-19-covid-combined.csv'

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
req = requests.get(url, headers=headers)
data_text = StringIO(req.text)

data = pd.read_csv(data_text)

In [2]:
def countryFig(country, fig, row, col, annotations, province=None, showlegend=False, linestyle='solid'):
    
    if province==None:
        countryData = data.loc[data['Country/Region']==country]
        label = country
    else:
        countryData = data.loc[data['Country/Region']==country].loc[data['Province/State']==province]
        label = '{} / {}'.format(province, country)
    
    dates     = countryData['Date']
    confirmed = countryData['Confirmed']
    recovered = countryData['Recovered']
    deaths    = countryData['Deaths']
    actives   = confirmed - recovered - deaths 
    days = range(len(confirmed))
    
    fig.add_trace(
    go.Scatter(x=dates, y=confirmed,
               marker = go.Marker(color= 'rgb(0, 0, 255)'),
               line={'dash':linestyle},
               name = "Total",
               showlegend=showlegend),        
    row=row, col=col)
    
    fig.add_trace(
    go.Scatter(x=dates, y=actives,
               marker = go.Marker(color= 'rgb(255, 0, 0)'),
               line={'dash':linestyle},
               name = "Active",
               showlegend=showlegend),
    row=row, col=col)
    
    annotations += [
        dict(
            text=r'<b>{}</b>'.format(label),
            showarrow=False,
            xref="paper",
            yref="paper",
            x=col-1,
            y=2-row)
    ]

In [6]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'scatter'}]])

# adding surfaces to subplots.
annotations = []
countryFig('Turkey', fig, 1, 1, annotations, showlegend=True)  #0,1
countryFig('Iran', fig, 1, 2, annotations)                     #1,1   
countryFig('Germany', fig, 2, 1, annotations)                  #0,0
countryFig('China', fig, 2, 2, annotations, province='Hubei')  #1,0

fig.update_layout(
    title_text=r'COVID-19: Confirmed Total Cases vs Active Cases',
    autosize=False,
    height=900,
    width=900,
    #margin=dict(l=65, r=50, b=65, t=90),
    annotations = annotations
    )

fig.show()

In [4]:
country, province = 'China', 'Hubei'

data.loc[data['Country/Region']==country].loc[data['Province/State']==province]

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths
6440,2020-01-22,China,Hubei,30.9756,112.2707,444.0,28.0,17.0
6441,2020-01-23,China,Hubei,30.9756,112.2707,444.0,28.0,17.0
6442,2020-01-24,China,Hubei,30.9756,112.2707,549.0,31.0,24.0
6443,2020-01-25,China,Hubei,30.9756,112.2707,761.0,32.0,40.0
6444,2020-01-26,China,Hubei,30.9756,112.2707,1058.0,42.0,52.0
...,...,...,...,...,...,...,...,...
6527,2020-04-18,China,Hubei,30.9756,112.2707,68128.0,63494.0,4512.0
6528,2020-04-19,China,Hubei,30.9756,112.2707,68128.0,63507.0,4512.0
6529,2020-04-20,China,Hubei,30.9756,112.2707,68128.0,63511.0,4512.0
6530,2020-04-21,China,Hubei,30.9756,112.2707,68128.0,63514.0,4512.0
